# Training

## Initial Setup

In [ ]:
!python --version

In [ ]:
!pip3 install torch torchvision torchaudio
import torch
import torchvision
# import torchaudio
print(torch.__version__)
print(torchvision.__version__)
# print(torchaudio.__version__)
from torchvision import transforms
import os
from pytorch_i3d import InceptionI3d
import videotransforms

!pip3 install av
import av
print(av.__version__)

# !pip3 install opencv-python
!pip3 install opencv-contrib-python

In [ ]:
!nvidia-smi
print(torch.cuda.is_available())

In [ ]:
print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))
if torch.cuda.is_available():
  dev = "cuda:0"
else:
  dev = "cpu"
device = torch.device(dev)
print(device)

## Training the Model with the data

In [ ]:
!python train_i3d.py -h

In [ ]:
!mkdir -p ./newmodels/
!python train_i3d.py -mode rgb -save_model ./newmodels/rgb_ucf -root ./rgb_frames/ -train_split ./ucf101.json -max_steps 100 -batch_size 1 -num_classes 101

In [ ]:
!mkdir -p ./newmodels/
!python train_i3d.py -mode flow -save_model ./newmodels/flow_ucf -root ./new_ucf101 -train_split ./ucf101.json -max_steps 3000 -batch_size 5 -num_classes 101

# Testing

# Model Prediction

In [ ]:
i3d = InceptionI3d(400, in_channels=3)
i3d.replace_logits(157)
i3d.load_state_dict(torch.load("./models/rgb_charades.pt"))
i3d.eval()
transform = transforms.Compose([videotransforms.CenterCrop(224)])  # Define the transformations to apply to the videos

In [ ]:
root = "rgb_frames"
vid = "workingAtLaptop"
start = 1
num_frames = len(os.listdir(os.path.join(root, vid)))
print(root, vid, start, num_frames)

In [ ]:
from charades_dataset import load_rgb_frames, video_to_tensor
print(os.path.join(root, vid))
imgs = load_rgb_frames(root, vid, start, num_frames)
cropTransform = transforms.Compose([videotransforms.CenterCrop(224)])
imgs = cropTransform(imgs)
input = video_to_tensor(imgs)
print(input.shape)
input = input[None] # batch size of 1
print(input.shape)

In [ ]:
# shape=(_BATCH_SIZE, _CHANNELS, _SAMPLE_VIDEO_FRAMES, _IMAGE_SIZE, _IMAGE_SIZE))
with torch.no_grad():
    prediction = i3d(input)

In [ ]:
classes = {}
with open("classes.txt", "r") as f:
    counter = 0
    lines = [line.strip() for line in f.readlines()]
    for line in lines:
        classes[counter] = line
        counter += 1    

In [ ]:
print(prediction.shape)
print(torch.max(prediction, dim=2)[0].argmax(1)) # first max to get max of all classes, then argmax to get index of max class
predicted_class = torch.max(prediction, dim=2)[0].argmax(1).item()
print(classes[predicted_class])